In [46]:
import requests
import zipfile
import os
import fnmatch

def download_file(url):
    dirpath = './data_files/'
    fname =  dirpath + url.split('/')[-1]
    r = requests.get(url, stream=True)
    with open (fname, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk:
                f.write(chunk)
                f.flush()
        return fname
    return False

def zip_extract(fname):
    dirpath = './data_files/'
    zfile = zipfile.ZipFile(fname)
    zfile.extractall(dirpath)

if __name__ == '__main__':
    #　保存ファイルの場所
    dirpath = './data_files/'
    
    # 福沢諭吉テキストurlのリストを作る！
    !grep 福沢,諭吉 aozora_word_list_utf8.csv > yukichi.csv
    text_url_list = [line.split(',')[45] for line in open('yukichi.csv')]
    
    #サーバーからデータをダウンロード && ファイルの展開
    for url in text_url_list:
        #fname = download_file(url)
        fname = './data_files/43029_ruby_23223.zip' #deback
        if fname:
            print('{} is download'.format(fname))
            zip_extract(fname)
        !sleep 1.0

    #すべて展開する
    # txt_list =[./data_files/kyoto_gakkono_ki.txt]
    txt_list = fnmatch.filter(os.listdir(dirpath), '*.txt')
    print(len(fname) == len(txt_list))
    
    #txt_list全体にdirpathを加える
    mapped_txt_list = map(lambda txt_path:dirpath+txt_path, txt_list)

./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip is download
./data_files/43029_ruby_23223.zip 

In [47]:
# 正規表現処理を関数化
# mapped_txt_listはgenerateで構成されているため一度使うと消える

import codecs
import re

def processing_regular_expression(text):
    # ヘッダ部分の除去
    text = re.split('-{5,}', text)[2]
    # フッタ部分の除去
    text = re.split('底本：', text)[0]
    # |の除去
    text = text.replace('｜', '')
    text = text.replace('|', '')
    # ルビの除去
    text = re.sub('《.+?》', '', text)
    # 入力注の除去
    text = re.sub('［＃.+?］', '', text)
    # 空行の除去
    text = re.sub('\n\n', '\n', text)
    text = re.sub('\u3000', '', text)
    # CR,LFの除去
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    return text

if __name__ == '__main__':
    all_sjis_text = "" # 処理済みテキストの格納

    # ファイル読込み、内部表現化
    for text in mapped_txt_list:
        f = codecs.open(text, "r", "sjis")
        sjis_text = f.read()
        f.close()    
        sjis_text = processing_regular_expression(sjis_text)
        all_sjis_text += sjis_text #テキストに保存
        
    # 整形結果確認   
    print(all_sjis_text[:100])
    print('\n\n')
    print(all_sjis_text[-100:])

明治五年申五月朔日、社友早矢仕氏とともに京都にいたり、名所旧跡はもとよりこれを訪うに暇あらず、博覧会の見物ももと余輩上京の趣意にあらず、まず府下の学校を一覧せんとて、知る人に案内を乞い、諸処の学校に行



難きほどのものなれば、貧富ともに勉むべきは学問にして、ただその教場をして仙境ならしめざること、吾々のつねに注意して怠らざるところなれば、学生諸氏もおのおの自から心してこの注意を空しゅうせしむるなかれ。


In [49]:
# mainは少なくする
from janome.tokenizer import Tokenizer

# Tokenneizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞のみの配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.surface for token in tokens if  token.part_of_speech.split(',')[0] in ['名詞']]

if __name__ == '__main__':
    # 全体のテキストを句点で区切った配列にする。
    sentence_list = all_sjis_text.split('。')

    # それぞれの文章を単語リストに変換
    word_list = [extract_words(sentence)for sentence in sentence_list]

    # 単語リストの結果を一部表示
    for word in word_list[0]:
        print(word)


明治
五
年
申
五月
朔日
社友
早
矢
氏
京都
名所旧跡
これ
暇
博覧
会
見物
もと
余
輩
上京
趣意
府下
学校
一覧
人
案内
乞い
諸処
学校
待遇
塾舎
講堂
ところ


In [51]:
#word2vecの使い方を調べる
#適切なハイパーパラメータを決める
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, size=100, alpha=0.01, window=5, min_count=3, iter=5)

In [52]:
#他にも適切な確認方法がないか調べる

# 結果の確認1
# 一つ一つの単語は100次元のベクトルになっています。 
# 「世間」のベクトル値を確認します。
print(model.__dict__['wv']['金'])

[ 1.97601598e-02  3.63006443e-03 -7.41567016e-02 -5.55482768e-02
  5.32528246e-03 -4.51689325e-02  4.87040030e-03 -1.04562258e-02
  8.87917057e-02  6.83920234e-02  1.50601760e-01  9.03183408e-03
  1.40525103e-01 -3.51149999e-02  7.06507340e-02  7.66725242e-02
 -6.78969920e-02 -8.79880339e-02  1.98746398e-02 -5.22150695e-02
  5.70226088e-02  1.20128412e-02  3.49547938e-02  4.12294157e-02
 -1.40134394e-02 -7.62532428e-02 -1.19028147e-02  8.68743435e-02
 -9.69357640e-02  1.36763668e-02 -4.21256162e-02 -3.80258225e-02
  6.91056103e-02  7.86385462e-02 -6.39136285e-02 -5.63166961e-02
  1.50873914e-01 -1.45532086e-01 -2.03896984e-02  1.37454227e-01
  1.63418353e-01  8.59540701e-02  2.25330871e-02 -3.82669299e-04
  3.29813361e-02 -8.64155591e-02  4.30007419e-03 -3.43955569e-02
  2.91459430e-02 -4.46299836e-02  3.20232734e-02  1.03695415e-01
  5.77519939e-04 -3.41199711e-02  1.38481655e-05 -2.09218338e-02
  9.49757099e-02 -5.86823747e-02 -6.14457158e-03  4.02374007e-02
 -1.64970122e-02  4.68434

In [168]:
#下に出てくる警告を調べできれば表示させないようにする

# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['命']) 
for item in ret:
    print(item[0], item[1])

どころ 0.4725894629955292
府 0.4475650489330292
上野 0.4118836522102356
強盗 0.41098856925964355
介 0.40858179330825806
受 0.4079661965370178
つもり 0.3988129496574402
役目 0.39686426520347595
暴政 0.3637733459472656
名目 0.3621255159378052


/Users/hamaguchikazuki/.pyenv/versions/3.6.5/envs/ppgVC/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
